## Evaluation

This notebook contains code for

1. Trainings process

2. Evaluation of P-Model

3. Evaluation of PF-Model

4. Time series plots for both, P and PF models

In order to keep this notebook clearly readable, some functions are outsourced in utils/

____

### Imports

In [ ]:
import pandas as pd
import pickle
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import config

#### Load models and history

In [ ]:
with open('data/models/history/P_optimized_3_history.pkl', 'rb') as f:
    history_P_opt = pickle.load(f)
with open('data/models/history/PF_optimized_2_history.pkl', 'rb') as f:
    history_PF_opt = pickle.load(f) 
with open('data/models/history/P_optimized_final_history.pkl', 'rb') as f:
    history_P = pickle.load(f)
with open('data/models/history/PF_optimized_final_history.pkl', 'rb') as f:
    history_PF_s = pickle.load(f) 

model_P = load_model('data/models/GRU_P_final.h5')
model_PF = load_model('data/models/GRU_PF_final.h5')

____

#### 1. Trainingsprocess

In [ ]:
from utils.plot_utils import plot_training_history

plot_training_history(history_P_opt, history_PF_opt)

#### 2. P-Model

In [ ]:
from utils.data_utils import data_loader
from utils.utils import train_test_val_data
import config

# load data
data = data_loader(config.columns_P)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

In [ ]:
from utils.utils import inverse_transform, get_monthly_metrics

# Vorhersagen auf den Testdaten machen
y_predicted = model_P.predict(X_test)
print("Shape predicted_test {}".format(y_predicted.shape))
print("Shape y_test {}".format(y_test.shape))

actual_values_p, predictions_p, X_test_actual_p = inverse_transform(y_predicted, y_test, X_test, scaler, config.columns_P, 1, "P_TOT")

In [ ]:
from utils.utils import inverse_transform, get_monthly_metrics
get_monthly_metrics(X_test_actual_p, actual_values_p, predictions_p)

In [ ]:
from utils.plot_utils import plot_predictions_vs_test, plot_predictions_vs_test_sequence
plot_predictions_vs_test(actual_values_p, predictions_p, X_test_actual_p, 99, multiplier=1.8, window_length=5, traces={"smoothing": False, "boosting": False})
plot_predictions_vs_test(actual_values_p, predictions_p, X_test_actual_p, 4, multiplier=1.8, window_length=5, traces={"smoothing": True, "boosting": True})

In [ ]:
plot_predictions_vs_test_sequence(X_test_actual_p, actual_values_p, predictions_p, 4093)

____

#### 3. PF-Model

In [ ]:
# load data
data = data_loader(config.columns_PF)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

In [ ]:
# Vorhersagen auf den Testdaten machen
y_predicted_PF = model_PF.predict(X_test)
print("Shape predicted_test {}".format(y_predicted_PF.shape))
print("Shape y_test {}".format(y_test.shape))

In [ ]:
actual_values_PF, predictions_PF, X_test_actual_PF = inverse_transform(y_predicted_PF, y_test, X_test, scaler, config.columns_PF, 1, "PF_TOT")

In [ ]:
monthly_metrics = get_monthly_metrics(X_test_actual_PF, actual_values_PF, predictions_PF)
monthly_metrics

#### 4. Time series plots

In [ ]:
from utils.plot_utils import plot_comparison
plot_comparison(
    actual_values_p, predictions_p, X_test_actual_p, 
    actual_values_PF, predictions_PF, X_test_actual_PF, 
    80)